# MiniGPT-4

In [1]:
%cd MiniGPT-4

/home/jupyter/opthollm/MiniGPT-4


### Imports

In [2]:
#@title Import
import argparse
import os
import random

import numpy as np
import torch
import torch.backends.cudnn as cudnn
import gradio as gr

from minigpt4.common.config import Config
from minigpt4.common.dist_utils import get_rank
from minigpt4.common.registry import registry
from minigpt4.conversation.multi_img_conversation import Chat, CONV_VISION

# imports modules for registration
from minigpt4.datasets.builders import *
from minigpt4.models import *
from minigpt4.processors import *
from minigpt4.runners import *
from minigpt4.tasks import *
import os

import argparse as argparse

### Helper Methods

In [3]:
#@title Helper Methods
def parse_args():
    parser = argparse.ArgumentParser(description="Demo")
    parser.add_argument("--cfg-path", required=True, help="path to configuration file.")
    parser.add_argument("--gpu-id", type=int, default=0, help="specify the gpu to load the model.")
    parser.add_argument("--num-beams", type=int, default=2, help="specify the gpu to load the model.")
    parser.add_argument("--temperature", type=int, default=0.9, help="specify the gpu to load the model.")
    parser.add_argument("--english", type=bool, default=True, help="chinese or english")
    parser.add_argument("--prompt-en", type=str, default="can you describe the current picture?", help="Can you describe the current picture?")
    parser.add_argument("--prompt-zh", type=str, default="你能描述一下当前的图片？", help="Can you describe the current picture?")
    parser.add_argument(
        "--options",
        nargs="+",
        help="override some settings in the used config, the key-value pair "
        "in xxx=yyy format will be merged into config file (deprecate), "
        "change to --cfg-options instead.",
    )
    args = parser.parse_args()
    return args


def setup_seeds(config):
    seed = config.run_cfg.seed + get_rank()

    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

    cudnn.benchmark = False
    cudnn.deterministic = True

def ask_model(chat_state, image, prompt, img_list):
    if image is not None: 
        chat_state.upload_img(image, chat_state, img_list)
    chat_state.ask(prompt, chat_state)
    llm_message = chat_state.answer(
        conv=chat_state,
        img_list=img_list,
        num_beams=args.num_beams,
        temperature=args.temperature,
        max_new_tokens=300,
        max_length=2000
    )[0]
    return llm_message

### Initialize Model

In [4]:
print('Initializing Chat')
#args = parse_args()
#args = preset_args
args = argparse.Namespace(cfg_path='eval_configs/minigpt4_eval.yaml', gpu_id=0, num_beams=2, temperature=0.9, english=True, prompt_en='can you describe the current picture?', prompt_zh='你能描述一下当前的图片？', options=None)
cfg = Config(args)

model_config = cfg.model_cfg
model_config.device_8bit = args.gpu_id
model_cls = registry.get_model_class(model_config.arch)
model = model_cls.from_config(model_config).to('cuda:{}'.format(args.gpu_id))

vis_processor_cfg = cfg.datasets_cfg.cc_sbu_align.vis_processor.train
vis_processor = registry.get_processor_class(vis_processor_cfg.name).from_config(vis_processor_cfg)
chat = Chat(model, vis_processor, device='cuda:{}'.format(args.gpu_id))
print('Initialization Finished')

print('Intializing Test')

Initializing Chat
Loading VIT
Loading VIT Done
Loading Q-Former
Loading Q-Former Done
Loading LLAMA

===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loading LLAMA Done
Load 4 training prompts
Prompt Example 
###Human: <Img><ImageHere></Img> Describe this image in detail. ###Assistant: 
Load BLIP2-LLM Checkpoint: pretrained_minigpt4.pth
Initialization Finished
Intializing Test


## Testing Example Image Consistency

In [8]:
# Select the example image file path 

minigpt4_example_img_path = 'StepTwoImageConsistencyImages/people_1.png' 
minigpt4_example_img_results = []

prompt = "Please describe the image"
chat_state = CONV_VISION.copy()

for i in range(5):
    chat_state = CONV_VISION.copy()
    llm_message = ask_model(chat_state, minigpt4_example_img_path, prompt, [])
    print(llm_message)
    minigpt4_example_img_results.append(llm_message)
   
minigpt4_example_img_results

The image shows a page from a manga or comic book. The panel depicts a fight scene between two characters, one with a serious expression and the other with a smirk on his face. The background is a cityscape with buildings and cars in the distance. The panel is drawn in a realistic style with attention to detail in the characters'faces and clothing. The panel is in black and white, with the characters and background in shading.
The image shows a page from a manga or anime comic book. The panel depicts a fight scene between two characters, one of whom is a dragon ball z character. The panel shows the characters in action, with one character delivering a punch to the other character's face. The panel is drawn in a manga or anime style, with exaggerated facial expressions and dynamic action lines. The panel is in black and white, with the characters and background in solid black and the action lines in white. The panel is part of a larger comic book or manga story, and the scene depicted i

["The image shows a page from a manga or comic book. The panel depicts a fight scene between two characters, one with a serious expression and the other with a smirk on his face. The background is a cityscape with buildings and cars in the distance. The panel is drawn in a realistic style with attention to detail in the characters'faces and clothing. The panel is in black and white, with the characters and background in shading.",
 "The image shows a page from a manga or anime comic book. The panel depicts a fight scene between two characters, one of whom is a dragon ball z character. The panel shows the characters in action, with one character delivering a punch to the other character's face. The panel is drawn in a manga or anime style, with exaggerated facial expressions and dynamic action lines. The panel is in black and white, with the characters and background in solid black and the action lines in white. The panel is part of a larger comic book or manga story, and the scene depi

In [9]:
# save results 

import pandas as pd

minigpt4_example_img_results_df = pd.DataFrame(minigpt4_example_img_results)
minigpt4_example_img_results_df.to_csv('minigpt4_example_img_consistency_test_results.csv')

## Testing Eye Image Consistency 

In [12]:
minigpt4_eye_img_path = 'StepTwoImageConsistencyImages/r1_Im069.png' 
minigpt4_eye_img_results = []

prompt = "Please diagnose the image as either glaucomatous or normal"

for i in range(5):
    chat_state = CONV_VISION.copy()
    llm_message = ask_model(chat_state, minigpt4_eye_img_path, prompt, [])
    print(llm_message)
    minigpt4_eye_img_results.append(llm_message)
    
minigpt4_eye_img_results

Based on the provided image, it appears to be a normal eye with no signs of glaucoma. The optic disc appears to be symmetrical and the cup-to-disc ratio is within normal limits. The neuroretinal rim appears intact and there are no signs of notching or defects. The blood vessels appear normal and there are no signs of vascular changes. Overall, the image appears to be normal.
Based on the provided image, it appears to be a normal eye with no signs of glaucoma. The optic disc appears to be symmetrical and the cup-to-disc ratio is within normal limits. The neuroretinal rim appears to be intact and there are no signs of notching or rim thinning. The blood vessels appear to be normal and there are no signs of vascular changes. Overall, the image appears to be normal.
Based on the provided image, it appears to be a normal eye with no signs of glaucoma. The optic disc appears to be symmetrical and the cup-to-disc ratio is within normal limits. The neuroretinal rim appears intact and there are

['Based on the provided image, it appears to be a normal eye with no signs of glaucoma. The optic disc appears to be symmetrical and the cup-to-disc ratio is within normal limits. The neuroretinal rim appears intact and there are no signs of notching or defects. The blood vessels appear normal and there are no signs of vascular changes. Overall, the image appears to be normal.',
 'Based on the provided image, it appears to be a normal eye with no signs of glaucoma. The optic disc appears to be symmetrical and the cup-to-disc ratio is within normal limits. The neuroretinal rim appears to be intact and there are no signs of notching or rim thinning. The blood vessels appear to be normal and there are no signs of vascular changes. Overall, the image appears to be normal.',
 'Based on the provided image, it appears to be a normal eye with no signs of glaucoma. The optic disc appears to be symmetrical and the cup-to-disc ratio is within normal limits. The neuroretinal rim appears intact and

In [13]:
# save results 

import pandas as pd

minigpt4_eye_img_results_df = pd.DataFrame(minigpt4_eye_img_results)
minigpt4_eye_img_results_df.to_csv('minigpt4_eye_img_consistency_test_results.csv')
minigpt4_eye_img_results_df.head()

,0
0,"Based on the provided image, it appears to be ..."
1,"Based on the provided image, it appears to be ..."
2,"Based on the provided image, it appears to be ..."
3,"Based on the provided image, it appears to be ..."
4,"Based on the provided image, it appears to be ..."
